In [2]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Activation
#country = 'India'
df = pd.read_csv("INFYtday.csv")
# df = pd.read_csv('NSE-TATAGLOBAL11.csv')
# df= pd.read_csv("NSE-TATA.csv")
df.index = df['Date']
#creating dataframe
data = df.sort_index(ascending=True, axis=0)
new_data = pd.DataFrame(index=range(0, len(df)), columns=['Date', 'Close'])
for i in range(0, len(data)):
    new_data['Date'][i] = data['Date'][i]
    new_data['Close'][i] = data['Close'][i]

#setting index
new_data.drop('Date', axis=1, inplace=True)
new_data.index = pd.to_datetime(df["Date"],format="%Y-%m-%d")

print("Total days in the dataset", len(new_data))


Total days in the dataset 1452


In [3]:
x = math.ceil((len(new_data)*80)/100)

train=new_data[:x]
test = new_data[x:]

##scale or normalize data as the data is too skewed

scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(train) 

train_scaled = scaler.transform(train)
test_scaled = scaler.transform(test)

In [4]:
seq_size = 50  ## number of steps (lookback)
n_features = 1 ## number of features. This dataset is univariate so it is 1
train_generator = TimeseriesGenerator(train_scaled, train_scaled, length = seq_size, batch_size=1)
print("Total number of samples in the original training data = ", len(train)) # 271
print("Total number of samples in the generated data = ", len(train_generator)) # 264 with seq_size=7

#Check data shape from generator
x,y = train_generator[10] 

Total number of samples in the original training data =  1162
Total number of samples in the generated data =  1112


In [5]:
test_generator = TimeseriesGenerator(test_scaled, test_scaled, length=seq_size, batch_size=1)
print("Total number of samples in the original training data = ", len(test)) # 14 as we're using last 14 days for test
print("Total number of samples in the generated data = ", len(test_generator)) # 7
#Check data shape from generator
x,y = test_generator[0]

Total number of samples in the original training data =  290
Total number of samples in the generated data =  240


In [ ]:
model = Sequential()
model.add(LSTM(150, activation='relu', return_sequences=True, input_shape=(seq_size, n_features)))
model.add(LSTM(64, activation='relu'))
model.add(Dense(64))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

model.summary()
print('Train...')
##########################

history = model.fit_generator(train_generator, 
                              validation_data=test_generator, 
                              epochs=50, steps_per_epoch=10)


#plot the training and validation accuracy and loss at each epoch
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
# plt.plot(epochs, loss, 'y', label='Training loss')
# plt.plot(epochs, val_loss, 'r', label='Validation loss')
# plt.title('Training and validation loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 50, 150)           91200     
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                55040     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 150,465
Trainable params: 150,465
Non-trainable params: 0
_________________________________________________________________
Train...


C:\Users\acer\anaconda3\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
10/10 [==============================] - 4s 195ms/step - loss: 0.0991 - val_loss: 0.0878
Epoch 2/50
10/10 [==============================] - 1s 157ms/step - loss: 0.0259 - val_loss: 0.0652
Epoch 3/50
10/10 [==============================] - 1s 161ms/step - loss: 0.0144 - val_loss: 0.0644
Epoch 4/50
10/10 [==============================] - 1s 147ms/step - loss: 0.0047 - val_loss: 3.8286
Epoch 5/50
10/10 [==============================] - 1s 148ms/step - loss: 0.0016 - val_loss: 129.5076
Epoch 6/50
10/10 [==============================] - 1s 149ms/step - loss: 0.0033 - val_loss: 41.3687
Epoch 7/50
10/10 [==============================] - 1s 148ms/step - loss: 0.0029 - val_loss: 1.4612
Epoch 8/50
10/10 [==============================] - 1s 148ms/step - loss: 0.0308 - val_loss: 361.2513
Epoch 9/50
10/10 [==============================] - 1s 146ms/step - loss: 0.0204 - val_loss: 0.4009
Epoch 10/50
10/10 [==============================] - 1s 149ms/step - loss: 0.0141 - val_loss: 0

In [ ]:
import math
from sklearn.metrics import mean_squared_error
math.sqrt(mean_squared_error(train,train_predict))

In [ ]:
prediction = [] #Empty list to populate later with predictions

current_batch = train_scaled[-seq_size:] #Final data points in train 
current_batch = current_batch.reshape(1, seq_size, n_features) #Reshape


In [ ]:
future = 7 #Days
for i in range(len(test) + future):
    current_pred = model.predict(current_batch)[0]
    prediction.append(current_pred)
    current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)

### Inverse transform to before scaling so we get actual numbers
rescaled_prediction = scaler.inverse_transform(prediction)

time_series_array = test.index  #Get dates for test data

#Add new dates for the forecast period
for k in range(0, future):
    time_series_array = time_series_array.append(time_series_array[-1:] + pd.DateOffset(1))

#Create a dataframe to capture the forecast data
df_forecast = pd.DataFrame(columns=["Close","predicted"], index=time_series_array)

df_forecast.loc[:,"predicted"] = rescaled_prediction[:,0]
df_forecast.loc[:,"Close"] = test["Close"]

#Plot
#df_forecast.plot(title="Predictions for next 7 days")
#plt.show()
df_forecast